In [13]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import HuggingFaceHub
from langchain.vectorstores import Chroma
from langchain.chains import ConversationalRetrievalChain

In [15]:
# Load the pdf file and split it into smaller chunks
loader = PyPDFLoader('./Prabhu Kiran Konda Resume.pdf')
documents = loader.load()

# Split the documents into smaller chunks 
text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=30)
texts = text_splitter.split_documents(documents)

In [17]:
embeddings = HuggingFaceEmbeddings()

config_sentence_transformers.json: 100%|██████████| 116/116 [00:00<00:00, 268kB/s]
README.md: 100%|██████████| 10.6k/10.6k [00:00<00:00, 9.02MB/s]
sentence_bert_config.json: 100%|██████████| 53.0/53.0 [00:00<00:00, 310kB/s]
config.json: 100%|██████████| 571/571 [00:00<00:00, 3.73MB/s]
pytorch_model.bin: 100%|██████████| 438M/438M [00:19<00:00, 22.2MB/s] 
tokenizer_config.json: 100%|██████████| 363/363 [00:00<00:00, 1.79MB/s]
vocab.txt: 100%|██████████| 232k/232k [00:00<00:00, 4.33MB/s]
tokenizer.json: 100%|██████████| 466k/466k [00:00<00:00, 2.72MB/s]
1_Pooling/config.json: 100%|██████████| 190/190 [00:00<00:00, 951kB/s]


In [18]:
db = Chroma.from_documents(texts, embeddings)
retriever = db.as_retriever(search_kwargs={'k': 2})

In [34]:
from langchain.llms import Ollama
from langchain_community.chat_models import ChatOllama
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

llm = Ollama(model="mistral:7b-instruct-q4_0", temperature=0, callbacks=[StreamingStdOutCallbackHandler()])

qa_chain = ConversationalRetrievalChain.from_llm(llm, retriever,return_source_documents=True)

In [35]:
import sys
chat_history = []

In [38]:
query = 'list all the projects mentioned in this resume'
result = qa_chain({'question': query, 'chat_history': chat_history})
print('Answer: ' + result['answer'] + '\n')
chat_history.append((query, result['answer']))

Number of requested results 2 is greater than number of elements in index 1, updating n_results = 1


Answer: 
1. AI-SaaS platform with SvelteKit and TailwindCSS, leveraging PostgreSQL for data storage.
2. Personal Portfolio + Markdown Blog with SvelteKit, TailwindCSS, TypeScript, and Vercel.
3. GFPGAN Image Restoration using Python, Streamlit, Docker, and HuggingFace.
4. STLF using Auto Encoders + RBF NN with Python, Tensorflow, Streamlit, and Weights & Biases.



In [39]:
chat_history

[('list all the publications mentioned in this resume',
  '\n1. A Platform Independent Web-Application for Short-Term Electric Power Load Forecasting on a 33/11 kV Substation Using Regression Model, Advances in Electrical & Electronics Engineering, Paper Link\n2. Weather Forecasting Using Radial Basis Function Neural Network in Warangal, India, MDPI Urban Science, Paper Link\n3. Active Power Load Data Dimensionality Reduction Using Autoencoder, Springer, Paper Link'),
 ('list all the projects mentioned in this resume',
  '\n1. AI-SaaS platform with SvelteKit and TailwindCSS, leveraging PostgreSQL for data storage.\n2. Personal Portfolio + Markdown Blog with SvelteKit, TailwindCSS, TypeScript, and Vercel.\n3. GFPGAN Image Restoration using Python, Streamlit, Docker, and HuggingFace.\n4. STLF using Auto Encoders + RBF NN with Python, Tensorflow, Streamlit, and Weights & Biases.')]